## Users 

### Upgrade date rank
- From Silver Users
- Rank the number of times a user switches from a nonpaying status to a paying status

### Upgrade Never
- From Silver User
- Users who have never paid on the platform

### Email Unsubs
- From Braze table
- A unsubscribe event before the report date (end of the month)

### Email Targeted
- From Braze table
- Received an email in this month

### Holdout (AFTER JULY 2021)
- From holdout table
- In current holdout cohort
- Excluses users who received an email in this month

### Email Engagers
- From Braze Marketing View
- Has an Adobe ID
- Non-Machine non-transactional email open before the report date (end of the month)

### All Cohorts
- From Braze Marketing View
- In Email Targeted or Holdout, and Silver Users
- Exclude Abandoned MAAs
- Exclude Unsubscribes

### Video Viewing
- From All Cohorts
- In Silver Video in the current month

In [ ]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Channel_KPI_Results_Feb_23` AS

WITH 
upgrade_date_rank AS (
    SELECT adobe_tracking_id
            , report_date
            , row_number() OVER(partition by adobe_tracking_id order by report_date ) as upgrade_row_number -- rank the number of times a user upgrade
    FROM       
            (
                SELECT adobe_tracking_id
                        , report_date
                        , paying_account_flag as paying_account_flag_today
                        , LAG(paying_account_flag,1) OVER ( partition by adobe_tracking_id order by report_date  ) as paying_account_flag_yestd -- paying flag yesterday
                FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
                --WHERE report_date between '2022-01-01' and '2022-07-10' -- don't add report_date since we need to distinguish upgrades from beginnning of time 
                ORDER BY 1,2    -- testing
            )
    WHERE paying_account_flag_today = 'Paying' AND paying_account_flag_yestd = 'NonPaying'
)
,upgrade_never AS (
    SELECT adobe_tracking_id
        , report_date
    FROM (
            SELECT adobe_tracking_id
                    , report_date
                    , sum(case when paying_account_flag = 'Paying' then 1 else 0 end) OVER(partition by adobe_tracking_id order by report_date ) as cumulative_nonpaying_num -- rank the number of times a user upgrade
            FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
        )
    WHERE cumulative_nonpaying_num = 0
)
,Email_Unsubs AS (
    SELECT DISTINCT adobe_tracking_id
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Unsubscribes' 
    AND event_date <= @report_end_date
)
,Email_Targeted AS ( -- everyone who have received emails in the month, and opened at least one email in the past
    SELECT DISTINCT adobe_tracking_id 
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Deliveries' 
    AND event_date BETWEEN @report_start_date and @report_end_date
)
--HOLDOUT 2.0, for months after July 2021
,Holdout AS ( 
    SELECT  distinct holdout.adobe_tracking_id
    FROM
    (
        SELECT  distinct TrackingId AS adobe_tracking_id -- distinct Hold_Out_Type_Current --cohort, count(distinct TrackingId)
        FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
        WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER)) -- get cohort name as first day of quarter
        AND Hold_Out_Type_Current = 'Owned Email Holdout'
        -- Exclude those who are assigned to Email Holdout but actually received emails 
    ) holdout
    LEFT JOIN
    (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
        WHERE event_name = 'Email Deliveries'
        AND event_date >= @report_start_date
        AND event_date <= @report_end_date 
    ) delivered -- Email Delivered ever
    ON holdout.adobe_tracking_id = delivered.adobe_tracking_id AND delivered.adobe_tracking_id is null
)
,All_Cohorts AS (
    -- email targetable and holdout cohort, users signed up after 2020/8/11
     SELECT distinct a.adobe_tracking_id as aid
     , cohort
     , user.account_type

    FROM         -- Include email engagers only: users who have at least 1 'Other Opens' in entire user histoy; exclude Oct 2022 Privacy email
     (SELECT DISTINCT identity 
            FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
            WHERE eventName = 'Email Opens' and machineOpen is null
            AND date(eventTimestamp) <= @report_end_date
            AND campaignName NOT IN ('TransactionalTermsOfUseEngaged20221028', 'TransactionalTermsOfUseNONEngaged20221028') -- Exclude email openers of Oct privacy email
        ) Email_Engagers --opened email at least once, using other open 

          INNER JOIN `nbcu-ds-sandbox-a-001.sl_sandbox.Braze_Id_Adobe_Id_Map` mapping
          ON mapping.bid = Email_Engagers.identity

          INNER JOIN
              (SELECT *, 'Email_Targeted' as cohort from Email_Targeted
                  UNION ALL
                  SELECT *, 'Holdout' as cohort from Holdout
              ) a
          ON a.adobe_tracking_id = mapping.aid
              
          --add attribute: account_type 
          INNER JOIN ( SELECT * FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` WHERE report_date = @report_end_date ) user
          ON a.adobe_tracking_id = user.adobe_tracking_id 
        

        -- for after 2021/july, email channel only, take out all abandon MAAs
        INNER JOIN 
        (SELECT DISTINCT adobe_tracking_id FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
            WHERE report_date = @report_end_date
            AND date_of_last_view IS NOT NULL 
        ) abandon_maa 
        ON a.adobe_tracking_id = abandon_maa.adobe_tracking_id

        -- exclude unsubscribed
        LEFT JOIN Email_Unsubs unsub
        ON a.adobe_tracking_id = unsub.adobe_tracking_id
        WHERE unsub.adobe_tracking_id is null
)
,Video_Viewing AS (
    SELECT 
        adobe_tracking_id
        , COUNT (DISTINCT CASE WHEN VIDEO.num_views_started =1 THEN video.adobe_tracking_id ELSE NULL END) AS Distinct_Content_Starts
        , SUM (VIDEO.num_views_started ) AS Total_Content_Starts
        , SUM(VIDEO.num_seconds_played_no_ads)/3600 AS Viewing_Time
        , COUNT(DISTINCT session_id) AS Distinct_Viewing_Sessions 
        , COUNT(DISTINCT(CASE WHEN (num_seconds_played_no_ads > CASE WHEN lower(consumption_type) = 'virtual channel' THEN 299 ELSE 0 END)
                and (num_views_started>0) THEN CASE WHEN (lower(consumption_type) = "shortform") THEN "Shortform"
                                                    WHEN lower(franchise) != 'other' THEN franchise ELSE display_name
                                                    END
                end)) as Repertoire_Pavo_Method
                
    FROM All_Cohorts a

    INNER JOIN  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` VIDEO
        ON VIDEO.adobe_tracking_id = a.aid
        AND adobe_date between @report_start_date AND @report_end_date
    GROUP BY 1
)

## KPI calcs

### Cohort Size 
- Number of unique users in the targeted/holdout cohort that is an email engager (defined as those with an email delivery in the last 4 months)
- Number of users, defined by All_Cohorts
- Sources: `All_Cohorts`

### Return Rate
- Percentage of the cohort with a video watch activity sometime in the reporting period.
- Num: Users with video watch activity (Silver Video) 
- Denom: Cohort Size
- Sources: `Video_Viewing`

### Usage
- Average viewing time of all users in the cohort
- Num: Sum of all user viewing time
- Denom: Cohort Size
- Sources: `Video_Viewing`

### Repertoire
- Average repertoire of all users in the cohort
- Num: Sum of all user repertoire
- Denom: Cohort Size
- Sources: `Video_Viewing`

### Viewing Sessions
- Average number of distinct viewing sessions of all users in the cohort
- Num: Sum of all distinct user viewing sessions
- Denom: Cohort Size
- Sources: `Video_Viewing`

### Viewing Winback Rate
- Percentage of the cohort who were past lapsing (last watch date between 30 and 90 days) sometime during the reporting period that had watched something in the reporting period
- Num: Users who watched something in the current month who were past lapsing (last watch date is between 30 and 90 days)
- Denom: Users in the month who are past lapsing (last watch date is between 30 and 90 days)
- Sources: `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`, `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO`

### Viewing Save Rate
- Percentage of the cohort who were past lapsing (last watch date between 15 and 29 days) sometime during the reporting period that had watched something in the reporting period
- Num: Users who watched something in the current month who were lapsing users (last watch date is betweeen 15 and 29 days)
- Denom: Users in the month who are lapsing users (last watch date is betweeen 15 and 29 days)
- Sources: `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`, `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO`

### Upgrade Rate
- Percentage of users with a nonpaying status sometime in the reporting period that then possess a paying account status with an entitlement shift sometime in the reporting period
- Num: Users with an entitlement change flag and a paying status in the current month
- Denom: Users with a nonpaying status in the current month
- Sources: `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`

### New Upgrade Rate
- Percentage of users who had never paid before who shifting to a paying status sometime in the reporting period
- Num: Users with their first change to paying status in the current month
- Denom: Users with no history of a paying status
- Sources: `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_never`, `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_date_rank`

### Paid Winback Rate
- Percentage of users who had paid before but with a nonpaying status in the current month that have no shifted to a paying status sometime in the reporting month
- Num: Users with a paying status in the current month who had a nonpaying status in the currrent month and previously been paying sometime in their history
- Denom: Users with a nonpaying status in the current month and previously been paying sometime in their history
- Sources: `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`, `upgrade_date_rank`, `upgrade_never`

### Cancellation Save Rate (prev. Paid Churn Save Rate)
- Percentage of users who turned off auto renew sometime in the reporting period who then turn it back on
- Num: Users with a paying account and auto renew turned on who had previously had a paying account and auto renew off in the current month
- Denom: Users with a paying account and auto renew off in the current month
- Sources: `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`

### Paid Churn Rate
- Percentage of users who have churned sometime in the reporting period who had been paying 
- Num: Users with a paid flag and a churn flag at the end of the month
- Denom: User with a paid flag at the end of the month
- Sources: `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`